<a href="https://colab.research.google.com/github/seph0916/seph-solo-machine/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A05%EB%8B%A8%EC%9B%90_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 교차 검증과 그리드 서치
# 테스트 세트에 맞는 모델을 만드는것이 아닌 일반적일때 잘 예측할수 있는 모델을 만들고싶음.
# 테스트 세트를 사용하지 않으면 모델이 과대,과소 적합인지 판단하기 어려움
# 테스트 세트를 사용하지 않고 측정하는 방법은 훈련세트를 또 나누는것-> 검증세트

In [2]:
# 와인 데이터 읽기
import pandas as pd
wine= pd.read_csv('https://bit.ly/wine-date')

In [3]:
# input data ,target data 나누기
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [5]:
# train data를 이용하여 또다시 훈련세트와 검증세트로 나눔
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [6]:
print(wine.shape,sub_input.shape,val_input.shape)

(6497, 4) (4157, 3) (1040, 3)


In [7]:
# train data중 80퍼센트(80퍼 중에 80퍼)에 해당하는 데이터를 통해 모델을 만들고 확인해보기
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))
# 예상했던것과 동일하게 훈련세트에만 과대적합되어있는 경향이 있음

0.9971133028626413
0.864423076923077


교차검증: 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복 , 그다음 이 점수를 평균하여 최종 검증 점수를 얻는 방법
(교차 검증 함수 cross_validate())

---

*   3-폴드 교차 검증 이란?
: 훈련세트를 3부분으로 나눠 교차검증을 수행하는것 ex) 훈련 셋 훈련 셋 검증 셋 ,훈련 셋 검증 셋 훈련 셋 , 검증 셋 훈련 셋 훈련 셋

In [8]:
# 교차 검증 함수 사용
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)
# fit time , score time : 모델을 훈련하는 시간과 검증하는 시간
# test score : 검증 폴드의 점수
# corss_validate() 함수는 훈련세트를 섞어 폴드를 나누지 않음

{'fit_time': array([0.01610518, 0.00880051, 0.00859332, 0.00873637, 0.00803256]), 'score_time': array([0.00176692, 0.00118995, 0.00106382, 0.0012002 , 0.00104189]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# StratifiedKFold 모델 만들어보기
from sklearn.model_selection import StratifiedKFold
# 앞서 수행한 교차검증 코드
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 10-폴드 교차 검증 수행 모델
# n_split 몇 폴드 교차 검증을 할지 정할수있음
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [12]:
# 사람의 개입없이 하이퍼 파라미터 (사용자 지정 파라미터) 튜닝 -> AutoML 이라 함

In [13]:
# Grid Search -> 하이퍼 파라미터 탐색 , 교차 검증을 한번에 수행하는 클래스
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
from sklearn.tree import DecisionTreeClassifier

In [14]:
# n_jobs -> cpu코어 수, -1로 지정하면 시스템에있는 모든코어사용
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [15]:
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
dt=gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [17]:
# 그리드 서치로 찾은 최적의 매개변수
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
# 각 매개변수에서 수행한 교차검증의 평균점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [28]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [29]:
# 결정트리에서의 최적 파라미터 찾아보기
# min_impurity_decrease : 불순도 감소 최소량, max_depth : 트리의 깊이, min_samples_split : 노드를 나누기 위한 최소 샘플수
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split': range(2,100,10)}

In [30]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [27]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [32]:
# gs.cv_result['mean_test_score'] 값중 max값  추출
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [36]:
# 그리드 서치 매개변수값이 수치일때 범위, 간격을 정하기가 어려울수 있다 + 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래걸림
# 이러한 단점을 보완하기 위해 랜덤서치를 사용
# 싸이파이 통해 2개의 확률분포 클래스 임포트
# uniform ->실숫값 뽑음 randint -> 정숫값 뽑음
from scipy.stats import uniform,randint

In [38]:
# rvs-> 랜덤으로 (a)개 만큼 뽑기
rgen=randint(0,10)
rgen.rvs(10)

array([2, 1, 0, 0, 7, 5, 7, 8, 7, 1])

In [40]:
# 0~9까지 해당하는 정수 갯수 0->102, 1->85 , ... , 9->74 느낌
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 90, 112, 104,  87, 104,  92,  94, 112, 102, 103]))

In [41]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.79279764, 0.66727395, 0.49547255, 0.41371218, 0.36178432,
       0.74410322, 0.10677841, 0.64475639, 0.20320023, 0.02864897])

In [42]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25),}

In [44]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,
                      n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3ca6c5ee00>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f3ca6c5cd60>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3ca6c5c4f0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f3ca6c5e050>},
                   random_state=42)

In [45]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [46]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [47]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
